In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
data_url = "https://www.hoopshype.com/salaries/players/"
driver = webdriver.Firefox()
driver.get(data_url)
html = driver.page_source
soup = BeautifulSoup(html)
for tag in soup.find_all('tr'):
    print(tag.prettify())

In [ ]:
data_url = "https://www.hoopshype.com/salaries/players/"
driver = webdriver.Firefox()
driver.get(data_url)
html = driver.page_source
soup = BeautifulSoup(html)

In [ ]:
year_buttons = driver.find_elements(By.CLASS_NAME,"ULq-fz__ULq-fz")
year_buttons[0].click()
all_years = driver.find_elements(By.CLASS_NAME,"_2qaD9L__2qaD9L")
# for yr in all_years:
#     yr.click()
#     time.sleep(3)

In [ ]:
data_urls = [yr.get_attribute("href") for yr in all_years]
wait = WebDriverWait(driver, 10) 
data_urls

In [ ]:
salary_df = pd.DataFrame()
row_idx = 0
MAX_RETRIES = 7
teams = {
    1:  "Atlanta Hawks",
    2:  "Boston Celtics",
    3:  "New Orleans Pelicans",
    4:  "Chicago Bulls",
    5:  "Cleveland Cavaliers",
    6:  "Dallas Mavericks",
    7:  "Denver Nuggets",
    8:  "Detroit Pistons",
    9:  "Golden State Warriors",
    10: "Houston Rockets",
    11: "Indiana Pacers",
    12: "Los Angeles Clippers",
    13: "Los Angeles Lakers",
    14: "Miami Heat",
    15: "Milwaukee Bucks",
    16: "Minnesota Timberwolves",
    17: "Brooklyn Nets",
    18: "New York Knicks",
    19: "Orlando Magic",
    20: "Philadelphia 76ers",
    21: "Phoenix Suns",
    22: "Portland Trail Blazers",
    23: "Sacramento Kings",
    24: "San Antonio Spurs",
    25: "Oklahoma City Thunder",
    26: "Utah Jazz",
    27: "Washington Wizards",
    28: "Toronto Raptors",
    29: "Memphis Grizzlies",
    53: "Charlotte Hornets",
}


for data_url in data_urls[:8]:
    driver = webdriver.Firefox()
    driver.get(data_url)
    html = driver.page_source
    soup = BeautifulSoup(html)
    table_pg_1 = soup.find_all('tr')

    headers_raw = table_pg_1[0].find_all(class_="RLrCiX__RLrCiX")
    headers = [header.text for header in headers_raw]
    page_num_count = 3
    while page_num_count != 2:
        retries = 0
        page_success = False

        while retries < MAX_RETRIES and not page_success:
            try:
                html = driver.page_source
                soup = BeautifulSoup(html)
                table = soup.find_all('tr')
                page_num = set(driver.find_elements(By.CLASS_NAME,"l98ZPp__l98ZPp")[0].text.split())
                page_num_count = len(page_num)
                print(page_num)

                for player in table[1:]:
                    try:
                        player_salaries = player.find_all(class_="RLrCiX__RLrCiX")
                        name = player.find(class_="_0cD6l-__0cD6l-").text
                        team_logo = player.find('img')
                        team_num_raw = team_logo.get('src').split("/")[7] 
                        team_num = int(team_num_raw[0]) if team_num_raw[1] == "." else int(team_num_raw[:2])
                    except Exception as e:
                        print(f"Error parsing player row: {e}")
                        retries += 1
                        print(f"Error processing page (attempt {retries}/{MAX_RETRIES}): {e}")
                        time.sleep(2)
                        if retries >= MAX_RETRIES:
                            print("Max retries reached for this page, skipping.")
                        
                    salary_df.loc[row_idx, "name"] = name
                    salary_df.loc[row_idx, "team"] = teams[team_num]
                    for i in range(len(player_salaries)):
                        salary_df.loc[row_idx, headers[i]] = player_salaries[i].text
                    row_idx += 1

                page_success = True

            except Exception as e:
                retries += 1
                print(f"Error processing page (attempt {retries}/{MAX_RETRIES}): {e}")
                time.sleep(2)
                if retries >= MAX_RETRIES:
                    print("Max retries reached for this page, skipping.")

        print(salary_df.shape)
        print(salary_df.iloc[-5:])

        try:
            buttons = driver.find_elements(By.CLASS_NAME, "hd3Vfp__hd3Vfp")
            buttons[1].click()
        except Exception as e:
            retries += 1
            print(f"Error processing page (attempt {retries}/{MAX_RETRIES}): {e}")
            time.sleep(2)
            if retries >= MAX_RETRIES:
                print("Max retries reached for this page, skipping.")


        time.sleep(2.5)

    driver.quit()
    salary_df.to_csv("data/00-raw/salary_data_raw.csv", index=False)
    time.sleep(10)